In [1]:
import pandas as pd
import numpy as np
import gzip
import re

In [19]:
# Path to the gzip file
data_folder = "../../BeerAdvocate/"
dataset_name = "ratings.csv"
# print the first 100 lines of the file
with open(data_folder + dataset_name, 'rt', encoding='utf-8') as file:
    for i in range(2225):
        file.readline()
    for i in range(3):
        print(repr(file.readline()))
        # Find the first line that is not a line skip and doesn't contain ":"
# with gzip.open(data_folder + dataset_name, 'rt', encoding='utf-8') as file:
#     lines = file.readlines()
#     for i, line in enumerate(lines):
#         if line.strip() and ": " not in line:
#             # Print the line with its neighboring lines
#             start = max(0, i - 1)
#             end = min(len(lines), i + 2)
#             for j in range(start, end):
#                 print(lines[j].strip())
#             break

# last_line = ""
# i=0
# with gzip.open(data_folder + dataset_name, 'rt', encoding='utf-8') as file:
#     while "330ml Bottle bought from the Vineyard" not in last_line:
#         last_line = file.readline()
#         i+=1
# print(f"Line {i}: {last_line}")

# print last_line showing all hidden characters
# print(repr(last_line))

import pandas as pd
import gzip
from tqdm import tqdm
import os

def parse_instance_chunk(chunk):
    # Split chunk into individual instances
    instances = chunk.strip().split('\n\n')
    data = []
    for instance in instances:
        instance_data = {}
        lines = instance.splitlines()
        current_line = ""
        
        for line in lines:
            # If the line contains ": ", we start a new key-value pair
            if ": " in line:
                # If there's a current line being accumulated, add it as a key-value pair
                if current_line:
                    key, value = current_line.split(": ", 1)
                    instance_data[key] = value
                
                # Start a new line accumulation with the current line
                current_line = line
            else:
                # If ": " is not found, concatenate this line to the previous one
                current_line += " " + line.strip()

        # Don't forget to add the last accumulated line as a key-value pair
        if current_line:
            key, value = current_line.split(": ", 1)
            instance_data[key] = value

        data.append(instance_data)
    return pd.DataFrame(data)

def read_and_store_chunks_to_csv(file_path, output_csv_path, instance_chunk_size=100):
    # Get the file size for progress estimation
    file_size = os.path.getsize(file_path)
    
    with gzip.open(file_path, 'rt') as f:
        instance_buffer = []
        first_chunk = True  # Track if this is the first chunk to write headers
        
        # Initialize tqdm progress bar with file size as the total length
        with tqdm(total=file_size, unit='B', unit_scale=True, desc="Processing file") as pbar:
            for line in f:
                instance_buffer.append(line)
                pbar.update(len(line))  # Update progress bar based on line length

                if line.strip() == "":
                    # Process the buffer if it reaches the chunk size limit
                    if len(instance_buffer) >= instance_chunk_size * 2:  # rough estimation for safety
                        chunk_text = ''.join(instance_buffer)
                        df_chunk = parse_instance_chunk(chunk_text)
                        
                        # Write to CSV, append if not the first chunk
                        df_chunk.to_csv(output_csv_path, mode='a', index=False, 
                                        header=first_chunk)
                        first_chunk = False  # Only write the header for the first chunk
                        instance_buffer = []  # Reset buffer for next chunk
            
            # Process any remaining lines in the buffer after the loop
            if instance_buffer:
                chunk_text = ''.join(instance_buffer)
                df_chunk = parse_instance_chunk(chunk_text)
                df_chunk.to_csv(output_csv_path, mode='a', index=False, header=first_chunk)

# data_folder = "../../BeerAdvocate/"
# dataset_name = "ratings.txt.gz"

# read_and_store_chunks_to_csv(data_folder + dataset_name, data_folder + "ratings.csv", instance_chunk_size=10000)

# data_folder = "../../RateBeer/"
# dataset_name = "ratings.txt.gz"

# read_and_store_chunks_to_csv(data_folder + dataset_name, data_folder + "ratings.csv", instance_chunk_size=10000)

# data_folder = "../../matched_beer_data/"
# dataset_name = "ratings_ba.txt.gz"

# read_and_store_chunks_to_csv(data_folder + dataset_name, data_folder + "ratings_ba.csv", instance_chunk_size=10000)

# data_folder = "../../matched_beer_data/"
# dataset_name = "ratings_rb.txt.gz"

# read_and_store_chunks_to_csv(data_folder + dataset_name, data_folder + "ratings_rb.csv", instance_chunk_size=10000)


'Alleycat\'s Amber Ale,78068,Breconshire Brewery,13090,English Bitter,4.2,1328266800,wl0307,wl0307.16869,3.0,3.5,3.5,3.0,3.5,3.27,"This beer is a contract brew produced by Breconshire Brewery for the Alleycat chain restaurants in Taiwan. Purchased and tasted at Alleycat’s branch in front of the Taipei Railway Station, Taipei City, the beer comes in a 500ml brown bottle as bottle-conditioned. Due to the fact that almost all of the bottles available were stored in the fridge at a very low temperature, I chose to have the only one stored at room temperature, but still during the pouring the yeast sediments managed to find their way into my straight imperial pint glass just as freely. NOTE: the back label specifies the hop variety used is Progress hops and the alc. is 4.2%abv., making me suspect if the beer has any connection with (e.g. inspired by?) the same brewery’s Golden Valley, as the latter also features Progress only at the same 4.2%abv.?A: pours a translucent light amber colour, c

In [2]:
def parse_review_block(block):
    review_data = {}
    for line in block.splitlines():
        if ": " in line:
            key, value = line.split(": ", 1)
            review_data[key.strip()] = value.strip()
    return review_data

def load_reviews_to_csv(file_path, output_csv_path, chunk_size=1000):
    chunk_data = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        review_block = []
        for line in file:
            if line.strip() == "":
                if review_block:
                    review_data = parse_review_block("\n".join(review_block))
                    chunk_data.append(review_data)
                    review_block = []

                    # When chunk reaches the specified size, append to CSV and clear memory
                    if len(chunk_data) >= chunk_size:
                        pd.DataFrame(chunk_data).to_csv(output_csv_path, mode='a', header=not pd.io.common.file_exists(output_csv_path), index=False)
                        chunk_data = []

            else:
                review_block.append(line)

        # Append remaining data after file read is complete
        if review_block:
            review_data = parse_review_block("\n".join(review_block))
            chunk_data.append(review_data)

        if chunk_data:
            pd.DataFrame(chunk_data).to_csv(output_csv_path, mode='a', header=not pd.io.common.file_exists(output_csv_path), index=False)


In [3]:
data_folder = '../../'
dataset_names = ['BeerAdvocate', 'RateBeer']
reviews_dfs = {}
for dataset_name in dataset_names:
    file_name = data_folder + dataset_name + '/reviews.txt.gz'
    output_name = data_folder + dataset_name + '/reviews.csv'
    load_reviews_to_csv(file_name, output_name)

In [40]:
number_lines = 100

data_folder = '../../'

dataset_names = ['BeerAdvocate', 'RateBeer']
beers_dfs = {}
breweries_dfs = {}
users_dfs = {}
ratings_dfs = {}

for dataset_name in dataset_names:
    # beers_df_iter = pd.read_csv(data_folder+dataset_name+'/beers.csv', chunksize=number_lines)
    # beers_df = pd.concat(beers_df_iter)
    # beers_dfs[dataset_name] = beers_df
    # print(dataset_name,"beers_df loaded")

    # breweries_df_iter = pd.read_csv(data_folder+dataset_name+'/breweries.csv', chunksize=number_lines)
    # breweries_df = pd.concat(breweries_df_iter)
    # breweries_dfs[dataset_name] = breweries_df
    # print(dataset_name,"breweries_df loaded")

    users_df_iter = pd.read_csv(data_folder+dataset_name+'/users.csv', chunksize=number_lines)
    users_df = pd.concat(users_df_iter)
    users_dfs[dataset_name] = users_df
    print(dataset_name,"users_df loaded")

    # ratings_df_iter = pd.read_csv(data_folder+dataset_name+'/ratings.csv', chunksize=number_lines, on_bad_lines='skip')
    # ratings_df = pd.concat(ratings_df_iter)
    # ratings_dfs[dataset_name] = ratings_df
    # print(dataset_name,"ratings_df loaded")
    pass




BeerAdvocate users_df loaded
RateBeer users_df loaded


In [4]:
# pickle the dataframe ratings_dfs["BeerAdvocate"]
# ratings_dfs["BeerAdvocate"].to_pickle(data_folder+'BeerAdvocate/ratings.pkl')

# load the pickle
ratings_dfs["BeerAdvocate"] = pd.read_pickle(data_folder+'BeerAdvocate/ratings.pkl')

In [5]:
print(ratings_dfs["BeerAdvocate"].head())

       beer_name  beer_id                               brewery_name  \
0          Régab   142544  Societe des Brasseries du Gabon (SOBRAGA)   
1  Barelegs Brew    19590       Strangford Lough Brewing Company Ltd   
2  Barelegs Brew    19590       Strangford Lough Brewing Company Ltd   
3  Barelegs Brew    19590       Strangford Lough Brewing Company Ltd   
4  Barelegs Brew    19590       Strangford Lough Brewing Company Ltd   

   brewery_id             style  abv        date        user_name  \
0       37262   Euro Pale Lager  4.5  1440064800          nmann08   
1       10093  English Pale Ale  4.5  1235127600      StJamesGate   
2       10093  English Pale Ale  4.5  1142247600          mdagnew   
3       10093  English Pale Ale  4.5  1101898800  helloloser12345   
4       10093  English Pale Ale  4.5  1093860000       cypressbob   

                 user_id  appearance  aroma  palate  taste  overall  rating  \
0         nmann08.184925        3.25   2.75    3.25   2.75      3.0    2.

In [3]:
ratings_df_iter = pd.read_csv(data_folder+dataset_name+'/ratings.csv', chunksize=number_lines)


In [15]:
ratings_df = pd.read_csv(data_folder+dataset_name)

ParserError: Error tokenizing data. C error: Expected 17 fields in line 2226, saw 18


In [21]:
import csv

with open(data_folder + dataset_name, newline='') as csvfile:
    reader = csv.reader(csvfile)
    for i, row in enumerate(reader):
        if i == 2225:  # 0-indexed, so line 2226 is at index 2225
            print(row)  # Print the problematic row
            break
print(f"Line {i}: {row}")
print(len(row))

["Alleycat's Amber Ale", '78068', 'Breconshire Brewery', '13090', 'English Bitter', '4.2', '1328266800', 'wl0307', 'wl0307.16869', '3.0', '3.5', '3.5', '3.0', '3.5', '3.27', 'This beer is a contract brew produced by Breconshire Brewery for the Alleycat chain restaurants in Taiwan. Purchased and tasted at Alleycat’s branch in front of the Taipei Railway Station, Taipei City, the beer comes in a 500ml brown bottle as bottle-conditioned. Due to the fact that almost all of the bottles available were stored in the fridge at a very low temperature, I chose to have the only one stored at room temperature, but still during the pouring the yeast sediments managed to find their way into my straight imperial pint glass just as freely. NOTE: the back label specifies the hop variety used is Progress hops and the alc. is 4.2%abv., making me suspect if the beer has any connection with (e.g. inspired by?) the same brewery’s Golden Valley, as the latter also features Progress only at the same 4.2%abv.?

In [43]:
# load matched_beer_data
# load data_folder+'matched_beer_data/beers.csv'
# it contains colums rb,rb,ba,ba
# beer_id,score,beer_id,score

dataset_name = 'matched_beer_data'

# beers_df_iter = pd.read_csv(data_folder + dataset_name + '/beers.csv', header=[0, 1], chunksize=number_lines)
# matched_beer_data_beers_df = pd.concat(beers_df_iter)
# print(dataset_name, "beers_df loaded")

# breweries_df_iter = pd.read_csv(data_folder + dataset_name + '/breweries.csv', header=[0, 1], chunksize=number_lines)
# matched_beer_data_breweries_df = pd.concat(breweries_df_iter)
# print(dataset_name, "breweries_df loaded")

# # ratins_df_iter = pd.read_csv(data_folder + dataset_name + '/ratings.csv', header=[0, 1], chunksize=number_lines)
# # matched_beer_data_ratings_df = pd.concat(ratins_df_iter)
# # print(dataset_name, "ratings_df loaded")

# users_approx_df_iter = pd.read_csv(data_folder + dataset_name + '/users_approx.csv', header=[0, 1], chunksize=number_lines)
# matched_beer_data_users_approx_df = pd.concat(users_approx_df_iter)
# print(dataset_name, "users_approx_df loaded")

users_df_iter = pd.read_csv(data_folder + dataset_name + '/users.csv', header=[0, 1], chunksize=number_lines)
matched_beer_data_users_df = pd.concat(users_df_iter)
print(dataset_name, "users_df loaded")

# file_path = data_folder + dataset_name + '/ratings_ba.txt.gz'
# chunks = []
# for chunk in parse_txtgz(file_path, num_lines=number_lines):
#     chunks.append(chunk)
# matched_beer_data_ratings_ba_df = pd.DataFrame(chunks)
# print(dataset_name, "ratings_ba_df loaded")

# file_path = data_folder + dataset_name + '/ratings_rb.txt.gz'
# chunks = []
# for chunk in parse_txtgz(file_path, num_lines=number_lines):
#     chunks.append(chunk)
# matched_beer_data_ratings_rb_df = pd.DataFrame(chunks)
# print(dataset_name, "ratings_rb_df loaded")

# file_path = data_folder + dataset_name + '/ratings_with_text_ba.txt.gz'
# chunks = []
# for chunk in parse_txtgz(file_path, num_lines=number_lines):
#     chunks.append(chunk)
# matched_beer_data_ratings_with_text_ba_df = pd.DataFrame(chunks)
# print(dataset_name, "ratings_with_text_ba_df loaded")

# file_path = data_folder + dataset_name + '/ratings_with_text_rb.txt.gz'
# chunks = []
# for chunk in parse_txtgz(file_path, num_lines=number_lines):
#     chunks.append(chunk)
# matched_beer_data_ratings_with_text_rb_df = pd.DataFrame(chunks)
# print(dataset_name, "ratings_with_text_rb_df loaded")

matched_beer_data users_df loaded


In [5]:
matched_beer_data_beers_df.head()

# print all columns (first level and second level)
for col in matched_beer_data_beers_df.columns:
    print(col)
    if isinstance(matched_beer_data_beers_df[col], pd.DataFrame):
        for col2 in matched_beer_data_beers_df[col].columns:
            print(col2)


('ba', 'abv')
('ba', 'avg')
('ba', 'avg_computed')
('ba', 'avg_matched_valid_ratings')
('ba', 'ba_score')
('ba', 'beer_id')
('ba', 'beer_name')
('ba', 'beer_wout_brewery_name')
('ba', 'brewery_id')
('ba', 'brewery_name')
('ba', 'bros_score')
('ba', 'nbr_matched_valid_ratings')
('ba', 'nbr_ratings')
('ba', 'nbr_reviews')
('ba', 'style')
('ba', 'zscore')
('rb', 'abv')
('rb', 'avg')
('rb', 'avg_computed')
('rb', 'avg_matched_valid_ratings')
('rb', 'beer_id')
('rb', 'beer_name')
('rb', 'beer_wout_brewery_name')
('rb', 'brewery_id')
('rb', 'brewery_name')
('rb', 'nbr_matched_valid_ratings')
('rb', 'nbr_ratings')
('rb', 'overall_score')
('rb', 'style')
('rb', 'style_score')
('rb', 'zscore')
('scores', 'diff')
('scores', 'sim')


In [6]:
# print beers_df columns
for col in beers_dfs['RateBeer'].columns:
    print(col)

# print beer_name and brewery_name head
print(beers_dfs['RateBeer'][['beer_name', 'brewery_name']].head())

beer_id
beer_name
brewery_id
brewery_name
style
nbr_ratings
overall_score
style_score
avg
abv
avg_computed
zscore
nbr_matched_valid_ratings
avg_matched_valid_ratings
                beer_name      brewery_name
0       33 Export (Gabon)           Sobraga
1     Castel Beer (Gabon)           Sobraga
2                   Régab           Sobraga
3  Ards Bally Black Stout  Ards Brewing Co.
4        Ards Belfast 366  Ards Brewing Co.


In [9]:
# print number of nans in ["rb"]["avg"]
col2 = "beer_id"
print("Number of NaNs in matched_beer_data_beers_df['rb'][col2]: ", matched_beer_data_beers_df['rb'][col2].isna().sum(),"/",len(matched_beer_data_beers_df['rb'][col2]))
print("Number of NaNs in matched_beer_data_beers_df['ba'][col2]: ", matched_beer_data_beers_df['ba'][col2].isna().sum(),"/",len(matched_beer_data_beers_df['ba'][col2]))
print("Number of NaNs in beers_dfs['RateBeer'][col2]: ", beers_dfs['RateBeer'][col2].isna().sum(),"/",len(beers_dfs['RateBeer'][col2]))
print("Number of NaNs in beers_dfs['BeerAdvocate'][col2]: ", beers_dfs['BeerAdvocate'][col2].isna().sum(),"/",len(beers_dfs['BeerAdvocate'][col2]))

Number of NaNs in matched_beer_data_beers_df['rb'][col2]:  0 / 45640
Number of NaNs in matched_beer_data_beers_df['ba'][col2]:  0 / 45640
Number of NaNs in beers_dfs['RateBeer'][col2]:  0 / 442081
Number of NaNs in beers_dfs['BeerAdvocate'][col2]:  0 / 280823


In [10]:
beer_id_mmatched_rb = 37923
# print row in matched_beer_data_beers_df["rb"] where beer_id == beer_id_mmatched_rb
print("matched")
print(matched_beer_data_beers_df["rb"][matched_beer_data_beers_df["rb"]["beer_id"] == beer_id_mmatched_rb])
print("RateBeer")
print(beers_dfs['RateBeer'][beers_dfs['RateBeer']["beer_id"] == beer_id_mmatched_rb])

matched
   abv   avg  avg_computed  avg_matched_valid_ratings  beer_id  \
0  4.8  2.79      2.923596                   2.923596    37923   

                   beer_name beer_wout_brewery_name  brewery_id  \
0  Strangford Lough Legbiter               Legbiter        4959   

       brewery_name  nbr_matched_valid_ratings  nbr_ratings  overall_score  \
0  Strangford Lough                         89           89           23.0   

                  style  style_score    zscore  
0  Golden Ale/Blond Ale         27.0 -0.698304  
RateBeer
     beer_id                  beer_name  brewery_id      brewery_name  \
289    37923  Strangford Lough Legbiter        4959  Strangford Lough   

                    style  nbr_ratings  overall_score  style_score   avg  abv  \
289  Golden Ale/Blond Ale           89           23.0         27.0  2.79  4.8   

     avg_computed    zscore  nbr_matched_valid_ratings  \
289      2.923596 -0.698304                         89   

     avg_matched_valid_ratings  


In [11]:
# print the columns that are not in both
print((set(matched_beer_data_beers_df["rb"].columns) - set(beers_dfs['RateBeer'].columns)) | (set(beers_dfs['RateBeer'].columns) - set(matched_beer_data_beers_df["rb"].columns)))
print((set(matched_beer_data_beers_df["ba"].columns) - set(beers_dfs['BeerAdvocate'].columns)) | (set(beers_dfs['BeerAdvocate'].columns) - set(matched_beer_data_beers_df["ba"].columns)))

{'beer_wout_brewery_name'}
{'beer_wout_brewery_name'}


In [12]:
comparison = matched_beer_data_beers_df["rb"]["beer_wout_brewery_name"] == matched_beer_data_beers_df["ba"]["beer_wout_brewery_name"]
print(matched_beer_data_beers_df[~comparison]["rb"]["beer_wout_brewery_name"])
print(matched_beer_data_beers_df[~comparison]["ba"]["beer_wout_brewery_name"])


8                                            Rockin Goose
12                                           Moovin On Up
16                                                   Citra
24                                             Wet Hop IPA
28                                          Spitting Llama
                               ...                        
45603                                  Pale Ale Bitterroot
45619    infused 1 lager tart ExperimentAle Czech w che...
45622                    Quarter Whiskey Cask Rye Kalamity
45627                               The Actress Bishop and
45634                                        Autumn Ode to
Name: beer_wout_brewery_name, Length: 10687, dtype: object
8                                            Rockin’ Goose
12                                            Moovin On Up
16                                               Citra IPA
24                                             Wet Ale Hop
28                                          LLama Spitti

# Merging beers

In [5]:
# remove "beer_wout_brewery_name" from matched_beer_data_beers_df["rb"] and ["ba"] because it is not in beers_dfs["RateBeer"] and ["BeerAdvocate"]
matched_beer_data_beers_df.drop(columns=["beer_wout_brewery_name"], level=1, inplace=True)

In [6]:
import pandas as pd
import numpy as np

def complete_matched_df(dfs, matched_df, id_col):
    # Step 1: Identify unmatched RateBeer beers
    if isinstance(id_col, tuple):
        # Create a combined identifier for RateBeer
        rb_combined = dfs["RateBeer"].apply(lambda row: tuple(row[list(id_col)]), axis=1)
        matched_combined = matched_df["rb"].apply(lambda row: tuple(row[list(id_col)]), axis=1)
        unmatched_rb = dfs["RateBeer"][~rb_combined.isin(matched_combined)]
    else:
        unmatched_rb = dfs["RateBeer"][~dfs["RateBeer"][id_col].isin(matched_df["rb"][id_col])]

    # Create DataFrame with 'rb' columns filled and 'ba' columns NaN
    unmatched_rb_df = pd.concat({
        'rb': unmatched_rb,
        'ba': pd.DataFrame(np.nan, index=unmatched_rb.index, columns=dfs["BeerAdvocate"].columns)
    }, axis=1)

    # Step 2: Identify unmatched BeerAdvocate beers
    if isinstance(id_col, tuple):
        # Create a combined identifier for BeerAdvocate
        ba_combined = dfs["BeerAdvocate"].apply(lambda row: tuple(row[list(id_col)]), axis=1)
        unmatched_ba = dfs["BeerAdvocate"][~ba_combined.isin(matched_combined)]
    else:
        unmatched_ba = dfs["BeerAdvocate"][~dfs["BeerAdvocate"][id_col].isin(matched_df["ba"][id_col])]

    # Create DataFrame with 'ba' columns filled and 'rb' columns NaN
    unmatched_ba_df = pd.concat({
        'rb': pd.DataFrame(np.nan, index=unmatched_ba.index, columns=dfs["RateBeer"].columns),
        'ba': unmatched_ba
    }, axis=1)

    # Step 3: Concatenate the matched and unmatched DataFrames
    completed_matched_df = pd.concat([matched_df, unmatched_rb_df, unmatched_ba_df], ignore_index=True)

    # Ensure columns are in the correct order (sorting by the original multi-index columns)
    completed_matched_df = completed_matched_df.sort_index(axis=1)

    # Step 4: Drop temporary columns from MultiIndex
    if isinstance(completed_matched_df.columns, pd.MultiIndex):
        # Create a mask for columns to keep
        mask = ~completed_matched_df.columns.get_level_values(0).str.contains('_temp_') & ~completed_matched_df.columns.get_level_values(1).str.contains('_temp_')
        completed_matched_df = completed_matched_df.loc[:, mask]
    else:
        # Handle single-level columns
        completed_matched_df = completed_matched_df.loc[:, ~completed_matched_df.columns.str.contains('_temp_')]

    return completed_matched_df


In [8]:
print(matched_beer_data_users_approx_df.columns)
print(matched_beer_data_users_df.columns)
print(len(matched_beer_data_users_approx_df))
print(len(matched_beer_data_users_df))

MultiIndex([(    'ba',          'joined'),
            (    'ba',        'location'),
            (    'ba',     'nbr_ratings'),
            (    'ba',     'nbr_reviews'),
            (    'ba',         'user_id'),
            (    'ba',       'user_name'),
            (    'ba', 'user_name_lower'),
            (    'rb',          'joined'),
            (    'rb',        'location'),
            (    'rb',     'nbr_ratings'),
            (    'rb',         'user_id'),
            (    'rb',       'user_name'),
            (    'rb', 'user_name_lower'),
            ('scores',             'sim')],
           )
MultiIndex([('ba',          'joined'),
            ('ba',        'location'),
            ('ba',     'nbr_ratings'),
            ('ba',     'nbr_reviews'),
            ('ba',         'user_id'),
            ('ba',       'user_name'),
            ('ba', 'user_name_lower'),
            ('rb',          'joined'),
            ('rb',        'location'),
            ('rb',     'nbr_ratin

In [32]:
ratings_dfs["BeerAdvocate"].columns

Index(['beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'abv',
       'date', 'user_name', 'user_id', 'appearance', 'aroma', 'palate',
       'taste', 'overall', 'rating', 'text', 'review'],
      dtype='object')

In [16]:
completed_beer_matched_df = complete_matched_df(beers_dfs, matched_beer_data_beers_df, "beer_id")
completed_brewery_matched_df = complete_matched_df(breweries_dfs, matched_beer_data_breweries_df, "id")
completed_user_matched_df = complete_matched_df(users_dfs, matched_beer_data_users_df, "user_id")
# completed_ratings_matched_df = complete_matched_df(ratings_dfs, matched_beer_data_ratings_df, ("beer_id","user_id"))


In [28]:
breweries_dfs["RateBeer"][breweries_dfs["RateBeer"]["id"] == 28180].shape[0]

1

In [31]:
beers_dfs["BeerAdvocate"]

,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
0,166064,Nashe Moskovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.7,NaN,NaN,0,NaN
1,166065,Nashe Pivovskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,3.8,NaN,NaN,0,NaN
2,166066,Nashe Shakhterskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.8,NaN,NaN,0,NaN
3,166067,Nashe Zhigulevskoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.0,NaN,NaN,0,NaN
4,166063,Zhivoe,39912,Abdysh-Ata (Абдыш Ата),Euro Pale Lager,0,0,NaN,NaN,NaN,4.5,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280818,19139,Kölsch Ale,885,Summit Station Restaurant & Brewery,Kölsch,3,3,2.71,NaN,NaN,5.0,2.713333,NaN,0,NaN
280819,19140,Nut Brown Ale,885,Summit Station Restaurant & Brewery,English Brown Ale,2,1,3.10,NaN,NaN,5.3,3.250000,NaN,0,NaN
280820,19146,Octoberfest,885,Summit Station Restaurant & Brewery,Märzen / Oktoberfest,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
280821,2805,Scotch Ale,885,Summit Station Restaurant & Brewery,Scotch Ale / Wee Heavy,0,0,NaN,NaN,NaN,7.0,NaN,NaN,0,NaN


In [14]:
completed_beer_matched_df

,ba_abv,ba_avg,ba_avg_computed,ba_avg_matched_valid_ratings,ba_ba_score,ba_beer_id,ba_beer_name,ba_brewery_id,ba_brewery_name,ba_bros_score,...,rb_brewery_name,rb_nbr_matched_valid_ratings,rb_nbr_ratings,rb_overall_score,rb_style,rb_style_score,rb_zscore,scores_diff,scores_sim,merged_beer_id
0,4.8,3.45,3.439867,3.504068,80.0,19827.0,Legbiter,10093.0,Strangford Lough Brewing Company Ltd,80.0,...,Strangford Lough,89.0,89.0,23.0,Golden Ale/Blond Ale,27.0,-0.698304,1.000000,1.0,0
1,6.0,3.86,3.888750,3.768333,NaN,20841.0,St. Patrick's Ale,10093.0,Strangford Lough Brewing Company Ltd,NaN,...,Strangford Lough,11.0,11.0,52.0,Irish Ale,79.0,0.005754,0.527141,1.0,1
2,4.2,3.56,3.556094,3.580417,82.0,20842.0,St. Patrick's Best,10093.0,Strangford Lough Brewing Company Ltd,90.0,...,Strangford Lough,74.0,74.0,26.0,Bitter,25.0,-0.862119,0.527141,1.0,2
3,4.8,3.96,3.960000,3.960000,NaN,22659.0,St. Patrick's Gold,10093.0,Strangford Lough Brewing Company Ltd,NaN,...,Strangford Lough,4.0,4.0,NaN,Amber Ale,NaN,-0.758491,0.527141,1.0,3
4,4.5,NaN,NaN,NaN,NaN,178681.0,Sheelin Stout,32848.0,The Sheelin Brewery,NaN,...,Sheelin,2.0,2.0,NaN,Mild Ale,NaN,-0.347271,1.000000,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677259,5.0,2.71,2.713333,NaN,NaN,19139.0,Kölsch Ale,885.0,Summit Station Restaurant & Brewery,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,677259
677260,5.3,3.10,3.250000,NaN,NaN,19140.0,Nut Brown Ale,885.0,Summit Station Restaurant & Brewery,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,677260
677261,NaN,NaN,NaN,NaN,NaN,19146.0,Octoberfest,885.0,Summit Station Restaurant & Brewery,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,677261
677262,7.0,NaN,NaN,NaN,NaN,2805.0,Scotch Ale,885.0,Summit Station Restaurant & Brewery,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,677262


In [18]:
# print lines where rb_id and ba_id are nan
print(completed_brewery_matched_df)

       merged_id    ba_id                    ba_location  \
0              0  10093.0               Northern Ireland   
1              1  32848.0               Northern Ireland   
2              2  40360.0               Northern Ireland   
3              3  40309.0               Northern Ireland   
4              4  41205.0               Northern Ireland   
...          ...      ...                            ...   
32707      32707    128.0                        Germany   
32708      32708   5023.0                          Aruba   
32709      32709   1573.0        United States, New York   
32710      32710  16186.0  United States, North Carolina   
32711      32711   1138.0        United States, Maryland   

                                    ba_name  ba_nbr_beers    rb_id  \
0      Strangford Lough Brewing Company Ltd           5.0   4959.0   
1                       The Sheelin Brewery           4.0  17616.0   
2               Walled City Brewing Company           6.0  24866.0   

In [17]:
# clean the completed dataframes by fixing the ids and making sure they are unique and compatible between beers and breweries

# flatten the multiindex
completed_beer_matched_df.columns = ['_'.join(col).strip() for col in completed_beer_matched_df.columns.values]
completed_brewery_matched_df.columns = ['_'.join(col).strip() for col in completed_brewery_matched_df.columns.values]
completed_user_matched_df.columns = ['_'.join(col).strip() for col in completed_user_matched_df.columns.values]

# create the column "merged_id" in completed_brewery_matched_df. It should be equal to the row index
completed_brewery_matched_df["merged_id"] = completed_brewery_matched_df.index

# create the column "merged_beer_id" in completed_beer_matched_df. It should be equal to the row index
completed_beer_matched_df["merged_beer_id"] = completed_beer_matched_df.index

# create the column "merged_user_id" in completed_user_matched_df. It should be equal to the row index
completed_user_matched_df["merged_user_id"] = completed_user_matched_df.index

# Move the merged_id columns to the first position
cols = completed_brewery_matched_df.columns.tolist()
cols.insert(0, cols.pop(cols.index('merged_id')))
completed_brewery_matched_df = completed_brewery_matched_df[cols]

cols = completed_beer_matched_df.columns.tolist()
cols.insert(0, cols.pop(cols.index('merged_beer_id')))
completed_beer_matched_df = completed_beer_matched_df[cols]

cols = completed_user_matched_df.columns.tolist()
cols.insert(0, cols.pop(cols.index('merged_user_id')))
completed_user_matched_df = completed_user_matched_df[cols]

# create the column "merged_brewery_id" in completed_beer_matched_df.
# If "ba_beer_id" is nan, it should be equal to "merged_id" in completed_brewery_matched_df where "rb_id" is equal to "rb_beer_id" in completed_beer_matched_df
# else ("rb_beer_id" is nan or no one is nan) it should be equal to "merged_id" in completed_brewery_matched_df where "ba_id" is equal to "ba_beer_id" in completed_beer_matched_df

# completed_beer_matched_df["merged_brewery_id"] = np.where(completed_beer_matched_df["ba_beer_id"].isna(),
#                                                             completed_brewery_matched_df.loc[completed_beer_matched_df["rb_brewery_id"].values, "merged_id"],
#                                                             completed_brewery_matched_df.loc[completed_beer_matched_df["ba_brewery_id"].values, "merged_id"])

# # check this last part is correct :
# def check_merged_brewery_id(row):
#     if np.isnan(row["ba_beer_id"]):
#         return completed_brewery_matched_df.loc[row["rb_brewery_id"], "merged_id"]
#     else:
#         return completed_brewery_matched_df.loc[row["ba_brewery_id"], "merged_id"]
# print((completed_beer_matched_df["merged_brewery_id"] == completed_beer_matched_df.apply(check_merged_brewery_id, axis=1)).all())

In [19]:
# a function to get the merged id from ba and rb ids
def get_merged_id(merged_df, ba_id_name, rb_id_name, merged_id_name, ba_id, rb_id):
    if np.isnan(ba_id):
        # return the merged id of the brewery with merged_df[rb_id_name] == rb_id
        return merged_df.loc[merged_df[rb_id_name] == rb_id, merged_id_name].values[0]
    else: # np.isnan(rb_id) or no one is nan
        # return the merged id of the brewery with merged_df[ba_id_name] == ba_id
        return merged_df.loc[merged_df[ba_id_name] == ba_id, merged_id_name].values[0]
    
# create the column "merged_brewery_id" in completed_beer_matched_df.

completed_beer_matched_df["merged_brewery_id"] = completed_beer_matched_df.apply(lambda row: get_merged_id(completed_brewery_matched_df, "ba_id", "rb_id", "merged_id", row["ba_brewery_id"], row["rb_brewery_id"]), axis=1)


In [32]:
# move the merged_brewery_id column to the second position
cols = completed_beer_matched_df.columns.tolist()
cols.insert(1, cols.pop(cols.index('merged_brewery_id')))
completed_beer_matched_df = completed_beer_matched_df[cols]


In [34]:
# ssave them as csv in data_folder/merged_dataset/
merged_dataset_folder = data_folder + "merged_dataset/"
completed_beer_matched_df.to_csv(merged_dataset_folder + "beers.csv", index=False)
completed_brewery_matched_df.to_csv(merged_dataset_folder + "breweries.csv", index=False)
completed_user_matched_df.to_csv(merged_dataset_folder + "users.csv", index=False)

In [74]:
matched_beer_data_ratings_with_text_rb_df

,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating,text
0,Ards Bally Black Stout,155699,Ards Brewing Co.,13538,Stout,4.6,1429178400,ciaranc,151109,3,7,2,8,17,3.70,"Bottle, gift from Aaron. Black, big head, lots..."
1,Ards Bally Black Stout,155699,Ards Brewing Co.,13538,Stout,4.6,1427796000,Rowlymo,198957,4,7,4,8,15,3.80,500ml Bottle in Bittles bar Belfast. Chocolate...
2,Ards Bally Black Stout,155699,Ards Brewing Co.,13538,Stout,4.6,1421665200,Don2711,285162,4,8,5,7,17,4.10,Great one and made very local to me. Bottle bo...
3,Ards Bally Black Stout,155699,Ards Brewing Co.,13538,Stout,4.6,1380621600,genegenie,224156,5,8,3,5,19,4.00,"Deep roasted aroma, good dark brown/black colo..."
4,Ards Bally Black Stout,155699,Ards Brewing Co.,13538,Stout,4.6,1371549600,kiwianer,88501,4,7,4,7,14,3.60,"The head is medium, the body black. It smells ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Farmageddon India Export Porter,263238,Farmageddon Brewing Co-Op,19507,Imperial Porter,5.2,1479812400,jamestulloch,138957,4,7,4,7,14,3.60,Bottle from The Beerhive. Dark brown/ruby body...
96,Farmageddon India Export Porter,263238,Farmageddon Brewing Co-Op,19507,Imperial Porter,5.2,1469268000,minutemat,265702,4,8,3,8,15,3.80,"500ml bottle. Pours black, large beige head. P..."
97,Farmageddon India Export Porter,263238,Farmageddon Brewing Co-Op,19507,Imperial Porter,5.2,1465639200,blackisle,267220,5,8,4,8,13,3.80,Bottle 50cl. Poured into a shaker. Pitch black...
98,Farmageddon India Export Porter,263238,Farmageddon Brewing Co-Op,19507,Imperial Porter,5.2,1457953200,BeerViking,73218,4,8,4,7,15,3.80,"CBR16, looks and smells great - black with a t..."


In [34]:
completed_brewery_matched_df

,merged_id,ba_id,ba_location,ba_name,ba_nbr_beers,rb_id,rb_location,rb_name,rb_nbr_beers,scores_diff,scores_sim
0,0,10093.0,Northern Ireland,Strangford Lough Brewing Company Ltd,5.0,4959.0,Northern Ireland,Strangford Lough,5.0,0.431275,0.889062
1,1,32848.0,Northern Ireland,The Sheelin Brewery,4.0,17616.0,Northern Ireland,Sheelin,2.0,0.526388,0.863596
2,2,40360.0,Northern Ireland,Walled City Brewing Company,6.0,24866.0,Northern Ireland,Walled City,3.0,0.527852,0.954183
3,3,40309.0,Northern Ireland,Ards Brewing Company,7.0,13538.0,Northern Ireland,Ards Brewing Co.,13.0,0.554395,0.896098
4,4,41205.0,Northern Ireland,Barrahooley Brewery,3.0,22304.0,Northern Ireland,Barrahooley Craft Brewery,4.0,0.602544,0.896205
...,...,...,...,...,...,...,...,...,...,...,...
32707,32707,128.0,Germany,Privatbrauerei Franz Joseph Sailer,6.0,NaN,NaN,NaN,NaN,NaN,NaN
32708,32708,5023.0,Aruba,Aruba Brewing,0.0,NaN,NaN,NaN,NaN,NaN,NaN
32709,32709,1573.0,"United States, New York",John Harvard’s Brewery & Ale House,491.0,NaN,NaN,NaN,NaN,NaN,NaN
32710,32710,16186.0,"United States, North Carolina",Triangle Brewing Company,20.0,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
# print all the row where the matching couple (beer_id, user_id) in matched_beer_data_ratings_with_text_ba_df and matched_beer_data_ratings_ba_df
print(matched_beer_data_ratings_with_text_rb_df[matched_beer_data_ratings_with_text_rb_df[["beer_id", "user_id"]].isin(matched_beer_data_ratings_rb_df[["beer_id", "user_id"]]).all(axis=1)])

                          beer_name beer_id               brewery_name brewery_id            style  abv        date        user_name user_id appearance aroma palate taste overall rating  \
0            Ards Bally Black Stout  155699           Ards Brewing Co.      13538            Stout  4.6  1429178400          ciaranc  151109          3     7      2     8      17   3.70   
1            Ards Bally Black Stout  155699           Ards Brewing Co.      13538            Stout  4.6  1427796000          Rowlymo  198957          4     7      4     8      15   3.80   
2            Ards Bally Black Stout  155699           Ards Brewing Co.      13538            Stout  4.6  1421665200          Don2711  285162          4     8      5     7      17   4.10   
3            Ards Bally Black Stout  155699           Ards Brewing Co.      13538            Stout  4.6  1380621600        genegenie  224156          5     8      3     5      19   4.00   
4            Ards Bally Black Stout  155699           A

In [94]:
print(len(reviews_dfs["RateBeer"]), len(reviews_dfs["BeerAdvocate"]))

1000 1000


# merging ratings

In [3]:
data_folder = '../../'
merged_dataset_folder = data_folder + "merged_dataset/"
# completed_beer_matched_df.to_csv(merged_dataset_folder + "beers.csv", index=False)
# completed_brewery_matched_df.to_csv(merged_dataset_folder + "breweries.csv", index=False)
# completed_user_matched_df.to_csv(merged_dataset_folder + "users.csv", index=False)

# load the merged dataframes
completed_beer_matched_df = pd.read_csv(merged_dataset_folder + "beers.csv")
completed_brewery_matched_df = pd.read_csv(merged_dataset_folder + "breweries.csv")
completed_user_matched_df = pd.read_csv(merged_dataset_folder + "users.csv")


/tmp/ipykernel_29366/2479762268.py:8: DtypeWarning: Columns (8,10,15,22,24,28) have mixed types. Specify dtype option on import or set low_memory=False.
  completed_beer_matched_df = pd.read_csv(merged_dataset_folder + "beers.csv")
/tmp/ipykernel_29366/2479762268.py:10: DtypeWarning: Columns (7,9,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  completed_user_matched_df = pd.read_csv(merged_dataset_folder + "users.csv")


In [4]:
# We will load the matched ratings then go throught the ratings in ba and rb and check if they are in the matched ratings or not

# load matched ratings
number_lines = 1000
dataset_name = 'matched_beer_data'
data_folder = '../../'

# Initialize lists to store chunks
ba_chunks = []
rb_chunks = []
ba_good_indices = []
rb_good_indices = []

# Process the 'ratings_ba.csv' file, keeping track of good row indices
for i, chunk in enumerate(pd.read_csv(data_folder + dataset_name + '/ratings_ba.csv', 
                                      chunksize=number_lines, 
                                      on_bad_lines='skip')):
    ba_chunks.append(chunk)
    ba_good_indices.extend(chunk.index)

# Process the 'ratings_rb.csv' file, keeping track of good row indices
for i, chunk in enumerate(pd.read_csv(data_folder + dataset_name + '/ratings_rb.csv', 
                                      chunksize=number_lines, 
                                      on_bad_lines='skip')):
    rb_chunks.append(chunk)
    rb_good_indices.extend(chunk.index)



In [5]:

# Find common good indices
common_good_indices = list(set(ba_good_indices).intersection(rb_good_indices))

# Concatenate chunks and filter to only include common good indices
matched_beer_data_ratings_ba_df = pd.concat(ba_chunks).loc[common_good_indices]
matched_beer_data_ratings_rb_df = pd.concat(rb_chunks).loc[common_good_indices]

print(dataset_name, "ratings_ba_df loaded with", len(matched_beer_data_ratings_ba_df), "rows")
print(dataset_name, "ratings_rb_df loaded with", len(matched_beer_data_ratings_rb_df), "rows")

matched_beer_data ratings_ba_df loaded with 857000 rows
matched_beer_data ratings_rb_df loaded with 857000 rows


In [6]:
# print the columns
print(matched_beer_data_ratings_ba_df.columns)
print(matched_beer_data_ratings_rb_df.columns)
print(len(matched_beer_data_ratings_ba_df))
print(len(matched_beer_data_ratings_rb_df))


Index(['beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'abv',
       'date', 'user_name', 'user_id', 'appearance', 'aroma', 'palate',
       'taste', 'overall', 'rating', 'text', 'review'],
      dtype='object')
Index(['beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'abv',
       'date', 'user_name', 'user_id', 'appearance', 'aroma', 'palate',
       'taste', 'overall', 'rating', 'text'],
      dtype='object')
857000
857000


In [7]:
# drop column "review" from matched_beer_data_ratings_ba_df
matched_beer_data_ratings_ba_df.drop(columns=["review"], inplace=True)

In [8]:
# matched_beer_data_ratings_ba_df and matched_beer_data_ratings_rb_df have the same (uni index) columns
# create a new dataframe such that the columns are ba_ and rb_

# create the new dataframe
matched_beer_data_ratings_df = pd.concat({
    'ba': matched_beer_data_ratings_ba_df,
    'rb': matched_beer_data_ratings_rb_df
}, axis=1)

# flatten the multiindex
matched_beer_data_ratings_df.columns = ['_'.join(col).strip() for col in matched_beer_data_ratings_df.columns.values]


In [10]:
matched_beer_data_ratings_df

,ba_beer_name,ba_beer_id,ba_brewery_name,ba_brewery_id,ba_style,ba_abv,ba_date,ba_user_name,ba_user_id,ba_appearance,...,rb_date,rb_user_name,rb_user_id,rb_appearance,rb_aroma,rb_palate,rb_taste,rb_overall,rb_rating,rb_text
0,Legbiter,19827,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.8,1417431600,Hellpop65,hellpop65.48993,NaN,...,1429178400,ciaranc,151109,3,7,2,8,17,3.7,"Bottle, gift from Aaron. Black, big head, lots..."
1,Legbiter,19827,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.8,1401357600,Latarnik,latarnik.52897,NaN,...,1427796000,Rowlymo,198957,4,7,4,8,15,3.8,500ml Bottle in Bittles bar Belfast. Chocolate...
2,Legbiter,19827,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.8,1393412400,RochefortChris,rochefortchris.697017,NaN,...,1421665200,Don2711,285162,4,8,5,7,17,4.1,Great one and made very local to me. Bottle bo...
3,Legbiter,19827,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.8,1392030000,OKCNittany,okcnittany.144868,NaN,...,1380621600,genegenie,224156,5,8,3,5,19,4.0,"Deep roasted aroma, good dark brown/black colo..."
4,Legbiter,19827,Strangford Lough Brewing Company Ltd,10093,English Pale Ale,4.8,1390647600,jaydoc,jaydoc.265507,NaN,...,1371549600,kiwianer,88501,4,7,4,7,14,3.6,"The head is medium, the body black. It smells ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856995,Night Before Christmas,77388,Twisted Manzanita Ales,23640,American Stout,7.4,1357642800,HalfFull,halffull.196628,3.50,...,1239271200,jrt,27614,4,8,5,7,17,4.1,"Rating#1Smelled great, and poured beautifully,..."
856996,Night Before Christmas,77388,Twisted Manzanita Ales,23640,American Stout,7.4,1354446000,NickMunford,nickmunford.100959,NaN,...,1135681200,matta,7084,5,8,4,9,18,4.4,Many thanks to rater FlacoAlto for sending me ...
856997,Night Before Christmas,77388,Twisted Manzanita Ales,23640,American Stout,7.4,1326538800,mactrail,mactrail.311921,3.50,...,1135335600,shrubber85,10924,4,6,4,7,16,3.7,Bottle. Primarily a musty aroma with fruit to...
856998,Night Fighter,104563,Twisted Manzanita Ales,23640,English Dark Mild Ale,4.0,1390129200,NickMunford,nickmunford.100959,NaN,...,1133694000,chriso,6832,3,8,4,8,15,3.8,"Bottle at London gathering, November 2005, cou..."


In [19]:
# add columns "merged_beer_id", "merged_brewery_id" and "merged_user_id" to matched_beer_data_ratings_df
# a function to get the merged id from ba and rb ids
def prepare_lookup_dicts(merged_df, ba_id_name, rb_id_name, merged_id_name):
    ba_id_to_merged_id = merged_df.set_index(ba_id_name)[merged_id_name].to_dict()
    rb_id_to_merged_id = merged_df.set_index(rb_id_name)[merged_id_name].to_dict()
    return ba_id_to_merged_id, rb_id_to_merged_id

def get_merged_id(ba_id_to_merged_id, rb_id_to_merged_id, ba_id, rb_id):
    if ba_id in [None, '', np.nan]:  # Check if ba_id is None, empty, or NaN
        return rb_id_to_merged_id.get(rb_id, np.nan)  # Use rb_id if ba_id is missing
    else:
        return ba_id_to_merged_id.get(ba_id, np.nan)  # Use ba_id otherwise
    
# Prepare lookup dictionaries for beer and brewery IDs
ba_beer_id_to_merged_id, rb_beer_id_to_merged_id = prepare_lookup_dicts(completed_beer_matched_df, "ba_beer_id", "rb_beer_id", "merged_beer_id")
ba_brewery_id_to_merged_id, rb_brewery_id_to_merged_id = prepare_lookup_dicts(completed_brewery_matched_df, "ba_id", "rb_id", "merged_id")
ba_user_id_to_merged_id, rb_user_id_to_merged_id = prepare_lookup_dicts(completed_user_matched_df, "ba_user_id", "rb_user_id", "merged_user_id")




In [20]:
# create the columns "merged_beer_id", "merged_brewery_id" and "merged_user_id" in matched_beer_data_ratings_df

matched_beer_data_ratings_df["merged_beer_id"] = matched_beer_data_ratings_df.apply(lambda row: get_merged_id(ba_beer_id_to_merged_id, rb_beer_id_to_merged_id, row["ba_beer_id"], row["rb_beer_id"]), axis=1)
matched_beer_data_ratings_df["merged_brewery_id"] = matched_beer_data_ratings_df.apply(lambda row: get_merged_id(ba_brewery_id_to_merged_id, rb_brewery_id_to_merged_id, row["ba_brewery_id"], row["rb_brewery_id"]), axis=1)
matched_beer_data_ratings_df["merged_user_id"] = matched_beer_data_ratings_df.apply(lambda row: get_merged_id(ba_user_id_to_merged_id, rb_user_id_to_merged_id, row["ba_user_id"], row["rb_user_id"]), axis=1)

# move the merged columns to the first position
cols = matched_beer_data_ratings_df.columns.tolist()
cols.insert(0, cols.pop(cols.index('merged_user_id')))
cols.insert(0, cols.pop(cols.index('merged_brewery_id')))
cols.insert(0, cols.pop(cols.index('merged_beer_id')))
matched_beer_data_ratings_df = matched_beer_data_ratings_df[cols]


In [31]:
# now let's write to a csv file
# first write matched_beer_data_ratings_df
# then go over "BeerAdvocate/ratings.csv" and convert and write the rows that are not in matched_beer_data_ratings_df to the csv file
# then go over "RateBeer/ratings.csv" and convert and write the rows that are not in matched_beer_data_ratings_df to the csv file

# save matched_beer_data_ratings_df
matched_beer_data_ratings_df.to_csv(merged_dataset_folder + "ratings.csv", index=False)


In [30]:
matched_beer_data_ratings_df = pd.read_csv(merged_dataset_folder + "ratings_matched.csv")

In [28]:
for i, chunk in enumerate(pd.read_csv(data_folder + 'BeerAdvocate/ratings.csv', chunksize=10000)):


In [36]:
# now let's write at the end of merged_dataset_folder + "ratings.csv" the rows that are not in matched_beer_data_ratings_df
import pandas as pd
from tqdm import tqdm

number_lines = 10000

# Get the total number of lines to estimate the number of chunks
total_lines = sum(1 for _ in open(data_folder + 'BeerAdvocate/ratings.csv'))
num_chunks = total_lines // number_lines + (1 if total_lines % number_lines > 0 else 0)

# Use tqdm to show progress on chunks
for i, chunk in enumerate(tqdm(pd.read_csv(data_folder + 'BeerAdvocate/ratings.csv', 
                                           chunksize=number_lines, 
                                           on_bad_lines='skip'), 
                              total=num_chunks, desc="Processing chunks")):
    # Keep only rows where (beer_id, user_id) is not in matched_beer_data_ratings_df as (ba_beer_id, ba_user_id)
    filtered_chunk = chunk[~chunk[["beer_id", "user_id"]].isin(matched_beer_data_ratings_df[["ba_beer_id", "ba_user_id"]]).all(axis=1)]
    
    # New DataFrame with columns of matched_beer_data_ratings_df
    new_chunk = pd.DataFrame(columns=matched_beer_data_ratings_df.columns)
    
    # Fill new_chunk columns with the values from filtered_chunk
    for col in filtered_chunk.columns:
        new_chunk["ba_" + col] = filtered_chunk[col]
    
    # Set the merged ids
    new_chunk["merged_beer_id"] = new_chunk.apply(lambda row: get_merged_id(ba_beer_id_to_merged_id, rb_beer_id_to_merged_id, row["ba_beer_id"], row["rb_beer_id"]), axis=1)
    new_chunk["merged_brewery_id"] = new_chunk.apply(lambda row: get_merged_id(ba_brewery_id_to_merged_id, rb_brewery_id_to_merged_id, row["ba_brewery_id"], row["rb_brewery_id"]), axis=1)
    new_chunk["merged_user_id"] = new_chunk.apply(lambda row: get_merged_id(ba_user_id_to_merged_id, rb_user_id_to_merged_id, row["ba_user_id"], row["rb_user_id"]), axis=1)
    
    # Save the new_chunk to the CSV file
    new_chunk.to_csv(merged_dataset_folder + "ratings.csv", mode='a', index=False, 
                     header=not pd.io.common.file_exists(merged_dataset_folder + "ratings.csv"))


Processing chunks:  91%|█████████ | 764/840 [13:05<01:18,  1.03s/it]


In [38]:
number_lines = 10000

# Get the total number of lines to estimate the number of chunks
total_lines = sum(1 for _ in open(data_folder + 'RateBeer/ratings.csv'))
num_chunks = total_lines // number_lines + (1 if total_lines % number_lines > 0 else 0)

# Use tqdm to show progress on chunks
for i, chunk in enumerate(tqdm(pd.read_csv(data_folder + 'RateBeer/ratings.csv', 
                                           chunksize=number_lines, 
                                           on_bad_lines='skip'), 
                              total=num_chunks, desc="Processing chunks")):
    # Keep only rows where (beer_id, user_id) is not in matched_beer_data_ratings_df as (rb_beer_id, rb_user_id)
    filtered_chunk = chunk[~chunk[["beer_id", "user_id"]].isin(matched_beer_data_ratings_df[["rb_beer_id", "rb_user_id"]]).all(axis=1)]
    
    # New DataFrame with columns of matched_beer_data_ratings_df
    new_chunk = pd.DataFrame(columns=matched_beer_data_ratings_df.columns)
    
    # Fill new_chunk columns with the values from filtered_chunk
    for col in filtered_chunk.columns:
        new_chunk["rb_" + col] = filtered_chunk[col]
    
    # Set the merged ids
    new_chunk["merged_beer_id"] = new_chunk.apply(lambda row: get_merged_id(ba_beer_id_to_merged_id, rb_beer_id_to_merged_id, row["ba_beer_id"], row["rb_beer_id"]), axis=1)
    new_chunk["merged_brewery_id"] = new_chunk.apply(lambda row: get_merged_id(ba_brewery_id_to_merged_id, rb_brewery_id_to_merged_id, row["ba_brewery_id"], row["rb_brewery_id"]), axis=1)
    new_chunk["merged_user_id"] = new_chunk.apply(lambda row: get_merged_id(ba_user_id_to_merged_id, rb_user_id_to_merged_id, row["ba_user_id"], row["rb_user_id"]), axis=1)

    # Save the new_chunk to the CSV file
    new_chunk.to_csv(merged_dataset_folder + "ratings.csv", mode='a', index=False, 
                     header=not pd.io.common.file_exists(merged_dataset_folder + "ratings.csv"))

Processing chunks:  96%|█████████▋| 688/713 [11:21<00:24,  1.01it/s]


In [44]:
print(len(users_dfs["RateBeer"]), len(users_dfs["BeerAdvocate"]))
print(len(matched_beer_data_users_df))

70174 153704
3020


Hey, here is the merged dataset : 

Some notes : 
* I kept both columns from both datasets. For example beers can have different names in the two datasets so I kept them in columns 'rb_name' and 'ba_name' and so on. This is particularly important for scores so i didn't want to choose how to merge score already.
* there are three datasets :ba, rb and matched (that contain rows that match ba and rb). By merging, I mean adding to the matched dataset the rows from ba and rb that are not already matched. I also added merged_ids that are compatible across the csvs to make it usable. if a row from ba is not matched for example, the columns of that row with 'rb_' will be filled with nans.
* Using this merged dataset might be a bad idea : I don't know if the matched dataset we are given is actually complete (ie for example all beers that are in both ba and rb are in matched) or not (I think probably not) So even if I handled the matched dataset, they might still be a lot of duplicates that we can't sort out. That is because the matched dataset is actually very small compared to ba and rb (example : number of users in ba : 153704, number of users in matched : only 3000, but it seems clear that more than 3000 people have accounts on both sites...) I think that the matched dataset is not here for us to merge the two datasets but to compare the two sites (for example how do people rate beers in rb compared to ba, how are the reviews etc) but I might be wrong.  